# eda025_osr
osrの処理方法検討

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 
import scipy.optimize as opt
import multiprocessing
pd.set_option('display.max_columns', 50)
import datetime
import json
import pathlib

In [2]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [3]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [4]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [5]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [6]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [7]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [8]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    return base_train, base_test, sample_sub, ground_truth

In [9]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

# osrファイル読み込み

In [ ]:
res = []
decoder = json.JSONDecoder()
with open('../input/osr/2020-05-14-US-MTV-1.json', 'r') as f:
    line = f.readline()
    while line:
        res.append(decoder.raw_decode(line))
        line = f.readline()

In [ ]:
msg_types = []
for l in res:
    msg = l[0]['msg_type']
    msg_types.append(msg)

In [ ]:
set(msg_types)

# m72

In [ ]:
res[110][0]

In [ ]:
output_df = pd.DataFrame()
xs = []
ys = []
zs = []

for l in res:
    msg = l[0]['msg_type']
    if msg == 72:
        xs.append(l[0]['x'])
        ys.append(l[0]['y'])
        zs.append(l[0]['z'])
output_df['x'] = xs
output_df['y'] = ys
output_df['z'] = zs
display(output_df)

In [ ]:
output_df['latDeg'] = ecef2lla(output_df['x'], output_df['y'], output_df['z'])[0]
output_df['lngDeg'] = ecef2lla(output_df['x'], output_df['y'], output_df['z'])[1]

output_df['phoneName'] = '_'

visualize_trafic(output_df, {'lat':output_df.at[0,'latDeg'], 'lon':output_df.at[0,'lngDeg']}, zoom=9)

In [ ]:
def parse_m72(osr_dict):
    xs = []
    ys = []
    zs = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 72:
            xs.append(l['x'])
            ys.append(l['y'])
            zs.append(l['z'])
    out_df = pd.DataFrame({'x':xs, 'y':ys, 'z':zs})
    return out_df

In [ ]:
def parse_m74(osr_dict):
    tows = []
    ns_residuals = []
    wns = []
    Ps = []
    L_is = []
    L_fs = []
    D_is = []
    D_fs = []
    cn0s = []
    locks = []
    flags = []
    sats = []
    codes = []
    
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 74:
            header_t = l['header']['t']
            tow = header_t['tow']
            ns_residual = header_t['ns_residual']
            wn = header_t['wn']
            
            for obs in l['obs']:
                tows.append(tow)
                ns_residuals.append(ns_residual)
                wns.append(wn)
                Ps.append(obs['P'])
                L_is.append(obs['L']['i'])
                L_fs.append(obs['L']['f'])
                D_is.append(obs['D']['i'])
                D_fs.append(obs['D']['f'])
                cn0s.append(obs['cn0'])
                locks.append(obs['lock'])
                flags.append(obs['flags'])
                sats.append(obs['sid']['sat'])
                codes.append(obs['sid']['code'])
            
    out_df = pd.DataFrame({'tow':tows, 'ns_residual':ns_residuals, 'wn':wns,
                           'P':Ps, 'L_i':L_is, 'L_f':L_fs, 'D_i':D_is, 'D_fs':D_fs,
                           'cn0':cn0s, 'lock':locks, 'flag':'flags', 'sat':sats, 'code':codes})
    
    assert (out_df['tow'] - out_df['tow'].shift(1)).min() >= 0, 'towが途中で減少'
    return out_df

In [ ]:
def parse_m137(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    tgd1s = []
    tgd2s = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 137:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            tgd1s.append(l['tgd1'])
            tgd2s.append(l['tgd2'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'tgd1':tgd1s, 'tgd2':tgd2s, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs})
    
    out_df = out_df[out_df['iodc']==out_df['iode']].copy()
    
    return out_df

In [ ]:
def parse_m138(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    tgds = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 138:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            tgds.append(l['tgd'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'tgd':tgds, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs})
    
    return out_df

In [ ]:
def parse_m139(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    gammas = []
    taus = []
    d_taus = []
    poss = []
    vels = []
    accs = []
    fcns = []
    iods = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 139:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            gammas.append(l['gamma'])
            taus.append(l['tau'])
            d_taus.append(l['d_tau'])
            poss.append(l['pos'])
            vels.append(l['vel'])
            accs.append(l['acc'])
            fcns.append(l['fcn'])
            iods.append(l['iod'])
                        
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 
                           'gamma':gammas, 'tau':taus, 'd_tau':d_taus, 'pos':poss, 'vel':vels, 'acc':accs, 'fcn':fcns, 'iod':iods})
    
    return out_df

In [ ]:
def parse_m141(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    bgd_e1e5as = []
    bgd_e1e5bs = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    sources = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 141:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            bgd_e1e5as.append(l['bgd_e1e5a'])
            bgd_e1e5bs.append(l['bgd_e1e5b'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            sources.append(l['source'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'bgd_e1e5a':bgd_e1e5as, 'bgd_e1e5b':bgd_e1e5bs, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs,
                           'source':sources})
    
    out_df = out_df[out_df['iodc']==out_df['iode']].copy()
    
    return out_df

In [ ]:
m72 = parse_m72(res)
m74 = parse_m74(res)
m138 = parse_m138(res)
m139 = parse_m139(res)
m141 = parse_m141(res)

In [ ]:
m74[['code', 'sat']].drop_duplicates().sort_values(['code', 'sat'])

In [ ]:
m138[['sat', 'code']].drop_duplicates()

In [ ]:
m139[['sat', 'code']].drop_duplicates()

# m72のすべての座標確認

In [ ]:
p = pathlib.Path('../input/osr')
files = list(p.glob('*.json'))

res = []
decoder = json.JSONDecoder()
df = pd.DataFrame()

for file in files:
    with open(file, 'r') as f:
        line = f.readline()
        while line:
            res.append(decoder.raw_decode(line))
            line = f.readline()
    tmp = parse_m72(res)
    tmp['collectionName'] = file.name
    df = df.append(tmp)

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df['latDeg'] = ecef2lla(df['x'], df['y'], df['z'])[0]
df['lngDeg'] = ecef2lla(df['x'], df['y'], df['z'])[1]

In [ ]:
df['phoneName'] = '_'

In [ ]:
f = files[17].name
visualize_collection(df, f)
print(f)

In [ ]:
df[df['collectionName']==f].reset_index()['latDeg'].plot()

めっちゃばらばらしている。。。  
m72にはtimestampないので、直近のxyzをm74に埋め込む処理必要？

In [ ]:
# m74を埋め込むm74処理
def parse_m74(osr_dict):
    tows = []
    ns_residuals = []
    wns = []
    Ps = []
    L_is = []
    L_fs = []
    D_is = []
    D_fs = []
    cn0s = []
    locks = []
    flags = []
    sats = []
    codes = []
    xs = []
    ys = []
    zs = []
    
    x = np.nan
    y = np.nan
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 72:
            x = l['x']
            y = l['y']
            z = l['z']
            
        if msg_type == 74:
            header_t = l['header']['t']
            tow = header_t['tow']
            ns_residual = header_t['ns_residual']
            wn = header_t['wn']
            
            for obs in l['obs']:
                tows.append(tow)
                ns_residuals.append(ns_residual)
                wns.append(wn)
                Ps.append(obs['P'])
                L_is.append(obs['L']['i'])
                L_fs.append(obs['L']['f'])
                D_is.append(obs['D']['i'])
                D_fs.append(obs['D']['f'])
                cn0s.append(obs['cn0'])
                locks.append(obs['lock'])
                flags.append(obs['flags'])
                sats.append(obs['sid']['sat'])
                codes.append(obs['sid']['code'])
                xs.append(x)
                ys.append(y)
                zs.append(z)
            
    out_df = pd.DataFrame({'tow':tows, 'ns_residual':ns_residuals, 'wn':wns,
                           'P':Ps, 'L_i':L_is, 'L_f':L_fs, 'D_i':D_is, 'D_fs':D_fs,
                           'cn0':cn0s, 'lock':locks, 'flag':'flags', 'sat':sats, 'code':codes,
                           'x':xs, 'y':ys, 'z':zs})
    
    return out_df

In [ ]:
res = []
decoder = json.JSONDecoder()
with open('../input/osr/2020-05-14-US-MTV-1.json', 'r') as f:
    line = f.readline()
    while line:
        res.append(decoder.raw_decode(line))
        line = f.readline()
m74 = parse_m74(res)

# 週をまたいだm74がないか確認

In [ ]:
m74['wn'].nunique()

In [ ]:
decoder = json.JSONDecoder()

for file in files:
    res = []
    with open(file, 'r') as f:
        line = f.readline()
        while line:
            res.append(decoder.raw_decode(line))
            line = f.readline()
    tmp = parse_m74(res)
    print(file.name, tmp['wn'].nunique())

名前が長いやつが０

In [ ]:
res = []
decoder = json.JSONDecoder()
with open('../input/osr/202103102340-202103110020.json', 'r') as f:
    line = f.readline()
    while line:
        res.append(decoder.raw_decode(line))
        line = f.readline()

In [ ]:
tmp = parse_m74(res)

名前が長いやつは74がなくて代わりに1600がある　　
→74と1600読めるようにしないといけない

In [ ]:
# m72, 74, 1600
def parse_obs(osr_dict):
    tows = []
    ns_residuals = []
    wns = []
    Ps = []
    L_is = []
    L_fs = []
    D_is = []
    D_fs = []
    cn0s = []
    locks = []
    flags = []
    sats = []
    codes = []
    xs = []
    ys = []
    zs = []
    
    x = np.nan
    y = np.nan
    z = np.nan
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 72:
            x = l['x']
            y = l['y']
            z = l['z']
            
        if (msg_type == 74) | (msg_type == 1600):
            header_t = l['header']['t']
            tow = header_t['tow']
            ns_residual = header_t['ns_residual']
            wn = header_t['wn']
            
            for obs in l['obs']:
                tows.append(tow)
                ns_residuals.append(ns_residual)
                wns.append(wn)
                Ps.append(obs['P'])
                L_is.append(obs['L']['i'])
                L_fs.append(obs['L']['f'])
                locks.append(obs['lock'])
                flags.append(obs['flags'])
                sats.append(obs['sid']['sat'])
                codes.append(obs['sid']['code'])
                xs.append(x)
                ys.append(y)
                zs.append(z)
            
    out_df = pd.DataFrame({'tow':tows, 'ns_residual':ns_residuals, 'wn':wns,
                           'P':Ps, 'L_i':L_is, 'L_f':L_fs, 
                           'lock':locks, 'flag':'flags', 'sat':sats, 'code':codes,
                           'x':xs, 'y':ys, 'z':zs})
    
    return out_df

In [ ]:
decoder = json.JSONDecoder()

for file in files:
    res = []
    with open(file, 'r') as f:
        line = f.readline()
        while line:
            res.append(decoder.raw_decode(line))
            line = f.readline()
    tmp = parse_obs(res)
    print(file.name, tmp['wn'].nunique())

1600も統合できた  
週をまたがっているものもなさそう

# エフェメリスとobsの結合方法

In [ ]:
# m72, 74, 1600
def parse_obs(osr_dict):
    tows = []
    ns_residuals = []
    wns = []
    Ps = []
    L_is = []
    L_fs = []
    D_is = []
    D_fs = []
    cn0s = []
    locks = []
    flags = []
    sats = []
    codes = []
    xs = []
    ys = []
    zs = []
    
    x = np.nan
    y = np.nan
    z = np.nan
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 72:
            x = l['x']
            y = l['y']
            z = l['z']
            
        if (msg_type == 74) | (msg_type == 1600):
            header_t = l['header']['t']
            tow = header_t['tow']
            ns_residual = header_t['ns_residual']
            wn = header_t['wn']
            
            for obs in l['obs']:
                tows.append(tow)
                ns_residuals.append(ns_residual)
                wns.append(wn)
                Ps.append(obs['P'])
                L_is.append(obs['L']['i'])
                L_fs.append(obs['L']['f'])
                locks.append(obs['lock'])
                flags.append(obs['flags'])
                sats.append(obs['sid']['sat'])
                codes.append(obs['sid']['code'])
                xs.append(x)
                ys.append(y)
                zs.append(z)
            
    out_df = pd.DataFrame({'tow':tows, 'ns_residual':ns_residuals, 'wn':wns,
                           'P':Ps, 'L_i':L_is, 'L_f':L_fs, 
                           'lock':locks, 'flag':'flags', 'sat':sats, 'code':codes,
                           'x':xs, 'y':ys, 'z':zs})
    
    return out_df

In [ ]:
decoder = json.JSONDecoder()

for file in files:
    res = []
    with open(file, 'r') as f:
        line = f.readline()
        while line:
            res.append(decoder.raw_decode(line))
            line = f.readline()

In [ ]:
obs = parse_obs(res)

In [ ]:
obs[['code', 'sat']].drop_duplicates().sort_values(['code', 'sat'])

codeからみるに、GPSとGallileoしかない。パースもそれだけでよさそう

In [ ]:
# 138
def parse_gps(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    tgds = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 138:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            tgds.append(l['tgd'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'tgd':tgds, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs})
    
    return out_df

In [ ]:
# 141
def parse_gal(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    bgd_e1e5as = []
    bgd_e1e5bs = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    sources = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 141:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            bgd_e1e5as.append(l['bgd_e1e5a'])
            bgd_e1e5bs.append(l['bgd_e1e5b'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            sources.append(l['source'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'bgd_e1e5a':bgd_e1e5as, 'bgd_e1e5b':bgd_e1e5bs, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs,
                           'source':sources})
    
    out_df = out_df[out_df['iodc']==out_df['iode']].copy()
    
    return out_df

In [ ]:
gps = parse_gps(res)

In [ ]:
gps[['code', 'sat']].drop_duplicates().sort_values(['code', 'sat'])

In [ ]:
gal = parse_gal(res)
gal[['code', 'sat']].drop_duplicates().sort_values(['code', 'sat'])

0, 9 : GPS  
14, 26 : GAL

# obsとepheの結合

In [ ]:
res = []
decoder = json.JSONDecoder()
with open('../input/osr/2020-05-14-US-MTV-1.json', 'r') as f:
    line = f.readline()
    while line:
        res.append(decoder.raw_decode(line))
        line = f.readline()

In [ ]:
obs = parse_obs(res)
gps = parse_gps(res)
gal = parse_gal(res)

In [ ]:
obs_gps = obs[obs['code'].isin([0,9])].copy()
obs_gal = obs[obs['code'].isin([14,26])].copy()

In [ ]:
gps_result = pd.DataFrame()
for sat in obs_gps['sat'].unique():
    obs_gps_tmp = obs_gps[obs_gps['sat']==sat].copy()
    gps_tmp = gps[gps['sat']==sat].copy()
    del gps_tmp['sat']
    del gps_tmp['code']
    del gps_tmp['wn']
    gps_result_tmp = pd.merge_asof(obs_gps_tmp, gps_tmp.sort_values('tow'), on='tow', direction='nearest')
    gps_result = gps_result.append(gps_result_tmp)

In [ ]:
gps_result

In [ ]:
gal_result = pd.DataFrame()
for sat in obs_gal['sat'].unique():
    obs_gal_tmp = obs_gal[obs_gal['sat']==sat].copy()
    gal_tmp = gal[gal['sat']==sat].copy()
    del gal_tmp['sat']
    del gal_tmp['code']
    del gal_tmp['wn']
    gal_result_tmp = pd.merge_asof(obs_gal_tmp, gal_tmp.sort_values('tow'), on='tow', direction='nearest')
    gal_result = gal_result.append(gal_result_tmp)

In [ ]:
result = pd.concat([gps_result, gal_result])

In [ ]:
result

# すべてのファイルを統合

In [204]:
# 138
def parse_gps(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    tgds = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 138:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            tgds.append(l['tgd'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'tgd':tgds, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs})
    
    return out_df

In [205]:
# 141
def parse_gal(osr_dict):
    sats = []
    codes = []
    tows = []
    wns = []
    uras = []
    fit_intervals = []
    valids = []
    health_bits = []
    bgd_e1e5as = []
    bgd_e1e5bs = []
    c_rss = []
    c_rcs = []
    c_ucs = []
    c_uss = []
    c_ics = []
    c_iss = []
    dns = []
    m0s = []
    eccs = []
    sqrtas = []
    omega0s = []
    omegadots = []
    ws = []
    incs = []
    inc_dots = []
    af0s = []
    af1s = []
    af2s = []
    iodes = []
    iodcs = []
    sources = []
    
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 141:
            sats.append(l['common']['sid']['sat'])
            codes.append(l['common']['sid']['code'])
            tows.append(l['common']['toe']['tow'])
            wns.append(l['common']['toe']['wn'])
            uras.append(l['common']['ura'])
            fit_intervals.append(l['common']['fit_interval'])
            valids.append(l['common']['valid'])
            health_bits.append(l['common']['health_bits'])
            bgd_e1e5as.append(l['bgd_e1e5a'])
            bgd_e1e5bs.append(l['bgd_e1e5b'])
            c_rss.append(l['c_rs'])
            c_rcs.append(l['c_rc'])
            c_ucs.append(l['c_uc'])
            c_uss.append(l['c_us'])
            c_ics.append(l['c_ic'])
            c_iss.append(l['c_is'])
            dns.append(l['dn'])
            m0s.append(l['m0'])
            eccs.append(l['ecc'])
            sqrtas.append(l['sqrta'])
            omega0s.append(l['omega0'])
            omegadots.append(l['omegadot'])
            ws.append(l['w'])
            incs.append(l['inc'])
            inc_dots.append(l['inc_dot'])
            af0s.append(l['af0'])
            af1s.append(l['af1'])
            af2s.append(l['af2'])
            iodes.append(l['iode'])
            iodcs.append(l['iodc'])
            sources.append(l['source'])
            
    out_df = pd.DataFrame({'sat':sats, 'code':codes, 'tow':tows, 'wn':wns, 'ura':uras, 'fit_interval':fit_intervals,
                           'valid':valids, 'health_bits':health_bits, 'bgd_e1e5a':bgd_e1e5as, 'bgd_e1e5b':bgd_e1e5bs, 
                           'c_rs':c_rss, 'c_rc':c_rcs, 'c_uc':c_ucs, 'c_us':c_uss, 'c_ic':c_ics, 'c_is':c_iss,
                           'dn':dns, 'm0':m0s, 'ecc':eccs, 'sqrta':sqrtas, 'omega0':omega0s, 'omegadot':omegadots,
                           'w':ws, 'inc':incs, 'inc_dot':inc_dots, 'af0':af0s, 'af1':af1s, 'af2':af2s, 'iode':iodes, 'iodc':iodcs,
                           'source':sources})
    
    out_df = out_df[out_df['iodc']==out_df['iode']].copy()
    
    return out_df

In [206]:
# m72, 74, 1600
def parse_obs(osr_dict):
    tows = []
    ts = []
    ns_residuals = []
    wns = []
    Ps = []
    L_is = []
    L_fs = []
    D_is = []
    D_fs = []
    cn0s = []
    locks = []
    flags = []
    sats = []
    codes = []
    xs = []
    ys = []
    zs = []
    
    x = np.nan
    y = np.nan
    z = np.nan
    for line in osr_dict:
        l = line[0]
        msg_type = l['msg_type']
        if msg_type == 72:
            x = l['x']
            y = l['y']
            z = l['z']
            
        if (msg_type == 74) | (msg_type == 1600):
            header_t = l['header']['t']
            tow = header_t['tow']
            ns_residual = header_t['ns_residual']
            wn = header_t['wn']
            
            for obs in l['obs']:
                tows.append(tow)
                ns_residuals.append(ns_residual)
                wns.append(wn)
                Ps.append(obs['P'])
                L_is.append(obs['L']['i'])
                L_fs.append(obs['L']['f'])
                locks.append(obs['lock'])
                flags.append(obs['flags'])
                sats.append(obs['sid']['sat'])
                codes.append(obs['sid']['code'])
                xs.append(x)
                ys.append(y)
                zs.append(z)
            
    out_df = pd.DataFrame({'tow':tows, 'ns_residual':ns_residuals, 'wn':wns,
                           'P':Ps, 'L_i':L_is, 'L_f':L_fs, 
                           'lock':locks, 'flag':'flags', 'sat':sats, 'code':codes,
                           'x':xs, 'y':ys, 'z':zs})
    
    return out_df

In [207]:
def merge_obs_ephe(osr_dict):
    obs = parse_obs(osr_dict)
    gps = parse_gps(osr_dict)
    gal = parse_gal(osr_dict)
    
    obs_gps = obs[obs['code'].isin([0,9])].copy()
    obs_gal = obs[obs['code'].isin([14,26])].copy()
    
    gps_result = pd.DataFrame()
    for sat in obs_gps['sat'].unique():
        obs_gps_tmp = obs_gps[obs_gps['sat']==sat].copy()
        gps_tmp = gps[gps['sat']==sat].copy()
        gps_tmp['tow'] = gps_tmp['tow'] * 1000
        gps_tmp['te'] = gps_tmp['tow']
        del gps_tmp['sat']
        del gps_tmp['code']
        del gps_tmp['wn']
        gps_result_tmp = pd.merge_asof(obs_gps_tmp.sort_values('tow'), gps_tmp.sort_values('tow'), on='tow', direction='backward')
        gps_result = gps_result.append(gps_result_tmp)
        
    gal_result = pd.DataFrame()
    for sat in obs_gal['sat'].unique():
        obs_gal_tmp = obs_gal[obs_gal['sat']==sat].copy()
        gal_tmp = gal[gal['sat']==sat].copy()
        gal_tmp['tow'] = gal_tmp['tow'] * 1000
        gal_tmp['te'] = gal_tmp['tow']
        del gal_tmp['sat']
        del gal_tmp['code']
        del gal_tmp['wn']
        gal_result_tmp = pd.merge_asof(obs_gal_tmp.sort_values('tow'), gal_tmp.sort_values('tow'), on='tow', direction='backward')
        gal_result = gal_result.append(gal_result_tmp)
        
    result = pd.concat([gps_result, gal_result])
    result = result.dropna(subset=['te'])
    return result

In [208]:
p = pathlib.Path('../input/osr')
files = list(p.glob('*.json'))
results = []

decoder = json.JSONDecoder()

for file in files:
    osrs = []
    with open(file, 'r') as f:
        line = f.readline()
        while line:
            osrs.append(decoder.raw_decode(line))
            line = f.readline()
    results.append(merge_obs_ephe(osrs))
total = pd.concat(results)
total = total.reset_index(drop=True)

# 距離計算

In [223]:
total

,tow,ns_residual,wn,P,L_i,L_f,lock,flag,sat,code,x,y,z,ura,fit_interval,valid,health_bits,tgd,c_rs,c_rc,c_uc,c_us,c_ic,c_is,dn,m0,ecc,sqrta,omega0,omegadot,w,inc,inc_dot,af0,af1,af2,iode,iodc,te,bgd_e1e5a,bgd_e1e5b,source
0,425092000,0,2105,1276672585,134179182,249,15,flags,1,14,-2.726437e+06,-4.347308e+06,3.775187e+06,3.12,14400.0,1.0,0.0,NaN,36.65625,211.50000,0.000002,0.000006,3.166497e-08,9.313226e-09,2.872262e-09,0.098379,0.000078,5440.601511,0.954246,-5.461299e-09,1.481552,0.982985,-2.260808e-10,-0.000857,-7.915446e-12,0.0,67.0,67.0,424200000.0,-2.095476e-09,-2.328306e-09,0.0
1,425092000,0,2105,1276672773,100198730,226,15,flags,1,26,-2.726437e+06,-4.347308e+06,3.775187e+06,3.12,14400.0,1.0,0.0,NaN,36.65625,211.50000,0.000002,0.000006,3.166497e-08,9.313226e-09,2.872262e-09,0.098379,0.000078,5440.601511,0.954246,-5.461299e-09,1.481552,0.982985,-2.260808e-10,-0.000857,-7.915446e-12,0.0,67.0,67.0,424200000.0,-2.095476e-09,-2.328306e-09,0.0
2,425093000,0,2105,1276651947,134177013,237,15,flags,1,14,-2.726437e+06,-4.347308e+06,3.775187e+06,3.12,14400.0,1.0,0.0,NaN,36.65625,211.50000,0.000002,0.000006,3.166497e-08,9.313226e-09,2.872262e-09,0.098379,0.000078,5440.601511,0.954246,-5.461299e-09,1.481552,0.982985,-2.260808e-10,-0.000857,-7.915446e-12,0.0,67.0,67.0,424200000.0,-2.095476e-09,-2.328306e-09,0.0
3,425093000,0,2105,1276652135,100197111,36,15,flags,1,26,-2.726437e+06,-4.347308e+06,3.775187e+06,3.12,14400.0,1.0,0.0,NaN,36.65625,211.50000,0.000002,0.000006,3.166497e-08,9.313226e-09,2.872262e-09,0.098379,0.000078,5440.601511,0.954246,-5.461299e-09,1.481552,0.982985,-2.260808e-10,-0.000857,-7.915446e-12,0.0,67.0,67.0,424200000.0,-2.095476e-09,-2.328306e-09,0.0
4,425094000,0,2105,1276631312,134174845,27,15,flags,1,14,-2.726437e+06,-4.347308e+06,3.775187e+06,3.12,14400.0,1.0,0.0,NaN,36.65625,211.50000,0.000002,0.000006,3.166497e-08,9.313226e-09,2.872262e-09,0.098379,0.000078,5440.601511,0.954246,-5.461299e-09,1.481552,0.982985,-2.260808e-10,-0.000857,-7.915446e-12,0.0,67.0,67.0,424200000.0,-2.095476e-09,-2.328306e-09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526599,422812000,0,2108,1321149852,103689439,165,15,flags,36,26,-2.725674e+06,-4.348353e+06,3.774538e+06,3.12,14400.0,1.0,0.0,NaN,-246.18750,131.15625,-0.000011,0.000010,3.725290e-09,1.303852e-08,2.445102e-09,2.102042,0.000161,5440.621628,2.662171,-5.264862e-09,-0.224027,0.991700,-1.325055e-10,0.000551,-4.945377e-12,0.0,63.0,63.0,421800000.0,3.725290e-09,4.423782e-09,0.0
2526600,422813000,0,2108,1321171688,138856032,201,15,flags,36,14,-2.725674e+06,-4.348353e+06,3.774538e+06,3.12,14400.0,1.0,0.0,NaN,-246.18750,131.15625,-0.000011,0.000010,3.725290e-09,1.303852e-08,2.445102e-09,2.102042,0.000161,5440.621628,2.662171,-5.264862e-09,-0.224027,0.991700,-1.325055e-10,0.000551,-4.945377e-12,0.0,63.0,63.0,421800000.0,3.725290e-09,4.423782e-09,0.0
2526601,422813000,0,2108,1321171982,103691176,118,15,flags,36,26,-2.725674e+06,-4.348353e+06,3.774538e+06,3.12,14400.0,1.0,0.0,NaN,-246.18750,131.15625,-0.000011,0.000010,3.725290e-09,1.303852e-08,2.445102e-09,2.102042,0.000161,5440.621628,2.662171,-5.264862e-09,-0.224027,0.991700,-1.325055e-10,0.000551,-4.945377e-12,0.0,63.0,63.0,421800000.0,3.725290e-09,4.423782e-09,0.0
2526602,422814000,0,2108,1321193817,138858358,153,15,flags,36,14,-2.725674e+06,-4.348353e+06,3.774538e+06,3.12,14400.0,1.0,0.0,NaN,-246.18750,131.15625,-0.000011,0.000010,3.725290e-09,1.303852e-08,2.445102e-09,2.102042,0.000161,5440.621628,2.662171,-5.264862e-09,-0.224027,0.991700,-1.325055e-10,0.000551,-4.945377e-12,0.0,63.0,63.0,421800000.0,3.725290e-09,4.423782e-09,0.0


In [298]:
idx = 0
mu = 3.986005 * 1e14
sqrta = total.at[idx, 'sqrta']
dn = total.at[idx, 'dn']
M0 = total.at[idx, 'm0']
t = total.at[idx, 'tow'] / 1000
te = total.at[idx, 'te'] / 1000
e = total.at[idx, 'ecc']
omega0 = total.at[idx, 'omega0']
omega_dot = total.at[idx, 'omegadot']
omega_e = 7.2921151467 * 1e-5
w = total.at[idx, 'w']
Crs = total.at[idx, 'c_rs']
Crc = total.at[idx, 'c_rc']
Cuc = total.at[idx, 'c_uc']
Cus = total.at[idx, 'c_us']
Cic = total.at[idx, 'c_ic']
Cis = total.at[idx, 'c_is']
i0 = total.at[idx, 'inc']
i_dot = total.at[idx, 'inc_dot']
x_v =  total.at[idx, 'x']
y_v = total.at[idx, 'y']
z_v = total.at[idx, 'z']

a = sqrta**2
a3 = a**3
n0 = sqrt(mu/a3)
n = n0 + dn
M = M0 + (n * (t - te))

E = M
for i in range(10):
    E = M + (e * sin(E))
    
v = atan((sqrt(1 - e**2) * sin(E)) / (cos(E) - e))

phi = v + w

u = phi + (Cuc * cos(2*phi)) + (Cus * sin(2*phi))
r = (a * (1-(e* cos(E)))) + (Crc * cos(2*phi)) + (Crs * sin(2*phi))
i = i0 + (i_dot * (t - te)) + (Cic * cos(2*phi)) + (Cis * sin(2*phi))

x = r * cos(u)
y = r * sin(u)

omega = omega0 + ((omega_dot - omega_e) * (t - te) - (omega_e * te))

x_sat = (cos(omega) * r * cos(u)) + (-sin(omega) * cos(i) * r * sin(u))
y_sat = (sin(omega) * r * cos(u)) + (cos(omega) * cos(i) * r * sin(u))
z_sat = sin(i) * r * sin(u)

In [307]:
idx = 0
mu = 3.986005 * 1e14
sqrta = total.at[idx, 'sqrta']
dn = total.at[idx, 'dn']
M0 = total.at[idx, 'm0']
t = total.at[idx, 'tow'] / 1000
te = total.at[idx, 'te'] / 1000
e = total.at[idx, 'ecc']
omega0 = total.at[idx, 'omega0']
omega_dot = total.at[idx, 'omegadot']
omega_e = 7.2921151467 * 1e-5
w = total.at[idx, 'w']
Crs = total.at[idx, 'c_rs']
Crc = total.at[idx, 'c_rc']
Cuc = total.at[idx, 'c_uc']
Cus = total.at[idx, 'c_us']
Cic = total.at[idx, 'c_ic']
Cis = total.at[idx, 'c_is']
i0 = total.at[idx, 'inc']
i_dot = total.at[idx, 'inc_dot']
x_v =  total.at[idx, 'x']
y_v = total.at[idx, 'y']
z_v = total.at[idx, 'z']

a = sqrta**2
a3 = a**3
n0 = sqrt(mu/a3)
n = n0 + dn
M = M0 + (n * (t - te))

E = M
for i in range(10):
    E = M + (e * sin(E))
    
r = a * (1 - cos(E))
v = atan2((sqrt(1 - e**2) * sin(E)), (cos(E)-e))
p = v + w
    
d_u = (Cuc * cos(2*p)) + (Cus * sin(2*p))
d_r = (Crc * cos(2*p)) + (Crs * sin(2*p))
d_i = (Cic * cos(2*p)) + (Cis * sin(2*p))

u = p + d_u
r = r + d_r
i = i0 + d_i + i_dot * (t - te)

x = r * cos(u)
y = r * sin(u)

omega = omega0 + ((omega_dot - omega_e) * (t - te) - (omega_e * te))

x_sat = x * cos(omega) - y * sin(omega) * cos(i)
y_sat = x * sin(omega) + y * cos(omega) * cos(i)
z_sat = y * sin(i)

In [308]:
sqrt(((x_sat - x_v)**2) + ((y_sat - y_v)**2) + ((z_sat - z_v)**2))

5912574.81549922

In [242]:
E = M
for i in range(10):
    E = M + (e * sin(E))
    print(E)

0.20898184585660198
0.20898184707840128
0.20898184707849404
0.20898184707849404
0.20898184707849404
0.20898184707849404
0.20898184707849404
0.20898184707849404
0.20898184707849404
0.20898184707849404


a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


KeyboardInterrupt: 